In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                    level=logging.INFO)

In [ ]:
import tweepy as tw

# define keys
consumer_key= 'gEJhQtgiIvxzNB50u4JPic8f4'
consumer_secret= 'iEfTG65lFX8cAzKJ4QIhJklvuh3tfWdaRAAWO3b17082dZaSiu'
access_token= '1369691334051852293-IGWGrIUKFY6rTwmrA5WD3YLkJrlUk5'
access_token_secret= '7ftmxYiYnso7PNkPOWOWKCkNrguFFMhwwPTHhQ6bFVvgG'
# authenticate and create api object
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
import re

# just see how iterate through first 3 user's timeline pages with cursor
for page in tw.Cursor(api.user_timeline,
                      id="indykaila", exclude_replies=True,
                      include_rts=False, tweet_mode='extended').pages(3):
    for status in page:
        # preprocess documents (remove links and punctuations) to raw texts
        print(status.full_text)
        print(status.full_text.lower().split())
        link_removed = re.sub(r'\bhttps:\S+', '', status.full_text.lower())
        punc_link_removed = re.sub(r'[–,-.!":]\D', ' ', link_removed)
        print(punc_link_removed.split(), '\n---')

# Corpora

In [ ]:
class MyTexts:
    """implement a generator object on a specific user timeline
    and preprocess (remove links and punctuations)
    to yield as row tokenized texts"""
    def __init__(self, pagination_num=3):
        self.pagination_num = pagination_num
        # cursor on user's timeline
        self.cursor = tw.Cursor(api.user_timeline, id="indykaila",
                              exclude_replies=True, include_rts=False,
                              tweet_mode='extended').pages(self.pagination_num)
    def __iter__(self):
        for page in self.cursor:
            for status in page:
                # cleaning: removing links and some punctuations
                link_removed = re.sub(r'\bhttps:\S+', '', status.full_text.lower())
                punc_link_removed = re.sub(r'[–,-.!":]\D', '', link_removed)
                yield punc_link_removed.split()

> Collect statistics about corpus, preprocess it and store training corpus

In [ ]:
from gensim import corpora

texts = MyTexts(3)
# colloct statistics about all tokens
dictionary = corpora.Dictionary(texts)
# preprocess: remove stop words and only once words from dictionary
stop_words = set('for of a an and the to in'.split())
stop_word_ids = [dictionary.token2id[stopword]
                 for stopword in stop_words
                 if stopword in dictionary.token2id]
once_word_ids = [tokenid
                 for tokenid, docfreq in dictionary.dfs.items()
                 if docfreq == 1]
dictionary.filter_tokens(stop_word_ids + once_word_ids)
dictionary.compactify()
print(dictionary.dfs)
# store training corpus in bow representation for later use
# prefer work with generators (corpus streaming) instead of creating lists of documents
# train_texts = [text for text in MyTexts(3)]
train_texts = MyTexts(3)
'''note that every token that removed from dictionary is a "blah" for doc2bow method
and wouldn't count in corpus bow representation
but I use them in train_texts just for see the full texts somewhere'''
# train_corpus_bow = [dictionary.doc2bow(text) for text in train_texts]
train_corpus_bow = (dictionary.doc2bow(text) for text in train_texts)
corpora.MmCorpus.serialize('train_corpus_bow.mm', train_corpus_bow)

2021-03-31 10:29:33,011 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-03-31 10:29:33,886 : INFO : built Dictionary(327 unique tokens: ['4', 'a', 'bayern', 'be', 'confirmed']...) from 28 documents (total 535 corpus positions)
2021-03-31 10:29:33,892 : INFO : storing corpus in Matrix Market format to train_corpus_bow.mm
2021-03-31 10:29:33,894 : INFO : saving sparse matrix to train_corpus_bow.mm


{2: 5, 1: 2, 4: 6, 0: 3, 3: 2, 5: 7, 17: 2, 7: 2, 12: 6, 9: 2, 10: 5, 13: 4, 16: 2, 18: 2, 15: 2, 6: 2, 8: 3, 14: 3, 19: 2, 11: 2, 21: 3, 23: 2, 22: 2, 20: 2, 27: 5, 24: 3, 26: 2, 25: 2, 29: 2, 28: 2, 30: 2, 33: 2, 31: 4, 34: 2, 35: 2, 32: 2, 36: 2, 37: 3, 38: 2, 39: 2, 40: 6, 42: 3, 43: 2, 41: 2, 46: 2, 45: 2, 44: 2, 48: 2, 49: 3, 47: 6, 50: 2, 53: 2, 52: 2, 51: 2, 56: 2, 54: 2, 58: 3, 55: 2, 57: 2, 59: 2, 61: 2, 62: 3, 60: 2, 63: 2, 64: 2, 65: 3}


2021-03-31 10:29:34,423 : INFO : PROGRESS: saving document #0
2021-03-31 10:29:35,434 : INFO : saved 28x66 matrix, density=9.578% (177/1848)
2021-03-31 10:29:35,436 : INFO : saving MmCorpus index to train_corpus_bow.mm.index


In [ ]:
dictionary.dfs

In [ ]:
train_texts = MyTexts(3)
for text in train_texts:
    print(text)

In [ ]:
train_corpus_bow = corpora.MmCorpus('train_corpus_bow.mm')
for doc in train_corpus_bow:
    print(doc)

# Train the Models with Training Corpus

In [ ]:
from gensim import models

# load training corpus in bow from disk
train_corpus_bow = corpora.MmCorpus('train_corpus_bow.mm')
# initialize a tfidf model with training corpus in bow: training
tfidf_model = models.TfidfModel(train_corpus_bow)
# tranform training corpus bow->tfidf
train_corpus_tfidf = tfidf_model[train_corpus_bow]
# train a LSI model with training corpus in tfidf
lsi_model = models.LsiModel(train_corpus_tfidf, id2word=dictionary, num_topics=2)
# tfidf->fold-in-lsi
train_corpus_lsi = lsi_model[train_corpus_tfidf]
train_corpus_lsi

In [ ]:
lsi_model.print_topics()

In [ ]:
train_texts = MyTexts(3)
for doc_lsi, as_text in zip(train_corpus_lsi, train_texts):
    print(doc_lsi, as_text)

[(0, -0.32233195687890615), (1, 0.09858006339441101)] ['bayern', 'munich', 'have', 'confirmed', 'robert', 'lewandowski', 'will', 'be', 'out', 'for', '4', 'weeks', 'with', 'a', 'knee', 'injury.']
[(0, -0.3347516749221574), (1, 0.24999741456584593)] ['sergio', 'aguero', 'has', 'been', 'advised', 'by', 'his', 'public', 'relations', 'team', 'not', 'to', 'move', 'to', '@chelseafc', '@fcbarcelona', 'are', 'leading', 'the', 'racetalks', 'to', 'start', 'next', 'week', 'with', 'the', 'spanish', 'club.']
[(0, -0.2576089869413192), (1, 0.011923011264475453)] ['if', 'you', 'want', 'to', 'make', 'some', 'extra', 'money', 'tomorrow', 'give', '@betciub', 'a', 'followthey', 'have', 'some', 'huge', 'winners', '💷', '#ad']
[(0, -0.23638720624149792), (1, 0.17809356494061007)] ['manchester', 'city', 'confirm', 'sergio', 'aguero', 'will', 'leave', 'club', 'this', 'summer']
[(0, -0.20411216861734688), (1, 0.293945284881936)] ['everyone', 'connected', 'to', '@ecomi_', 'deserve', 'all', 'the', 'praise', 'they